In [13]:
import numpy as np
import sklearn

from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import ShuffleSplit, GridSearchCV, train_test_split
from sklearn import svm, metrics
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids


In [14]:
# Importamos los datos

X=np.load("./datos_mfcc_pca99_X.npy")
y=np.load("./datos_mfcc_pca99_y.npy")

In [15]:
#Reducimos el conjunto de entrenamiento al 50% usando un método no supervisado. En este caso, un C-means con el que pretendemos
# quedarnos con los centros de los clusters que aparezcan, que serán los puntos más representativos.

c = 6             # número de clases
nc = 120          # muestras por clase
n= 720           # número de muestras totales

X_reduc=[]
y_reduc=np.int16(np.kron(np.arange(c),np.ones(nc//2)))

for i in range(0,n,nc):
    k_medoids=KMedoids(n_clusters=nc//2)
    k_medoids.fit(X[i:(i+nc)])
    centros=k_medoids.cluster_centers_
    X_reduc.append(centros)

In [16]:
print(X_reduc[0].shape)
# Reshapeamos las dimensiones de X_reduc para tenerlo todo en una matriz dónde cada fila será una muestra y cada columna una 
# característica. La matriz estará ordenada por clases. 
# Para reshapear, antes habrá que convertir a array.


X_reduc=np.array(X_reduc).reshape((360,464))
print(X_reduc.shape)

(60, 464)
(360, 464)


In [17]:
# Ahora ya podemos dividir nuestro conjunto reducido en entrenamiento y test, y probar algunos métodos Kernel.

Xtr, Xts, ytr, yts = train_test_split(X_reduc, y_reduc, test_size=0.3, shuffle=True, random_state=0)


In [18]:
# Aplicamos directamente un clasificador SVC con los parámetros por defecto y calculamos algunas métricas para ver cómo funciona

clf=svm.SVC(verbose=True)
clf.fit(Xtr,ytr)

score_tr = clf.score(Xtr,ytr)
score_ts = clf.score(Xts,yts)
preds_train=clf.predict(Xtr)
preds_test = clf.predict(Xts)

print('')
print('OA train %0.2f' % score_tr)
print('Kappa train %0.2f' % metrics.cohen_kappa_score(ytr,preds_train))
print('OA test %0.2f' % score_ts)
print('Kappa test %0.2f' % metrics.cohen_kappa_score(yts,preds_test))


[LibSVM]
OA train 0.79
Kappa train 0.75
OA test 0.59
Kappa test 0.52


In [19]:
# Matriz de confusión

CM=metrics.confusion_matrix(yts,preds_test)
print(CM)

[[ 8  2  1  1  2  3]
 [ 0 14  0  0  0  6]
 [ 0  0 11  4  2  3]
 [ 2  0  6 12  0  2]
 [ 3  0  0  0  9  6]
 [ 0  0  0  0  1 10]]


Observamos que con los parámetros por defecto, obtenemos unos resultados bastante buenos en entrenamiento y un acierto bastante decente en test. 

Además, vemos que la matriz de confusión está bastante balanceada, por lo que tiene buena pinta este clasificador para nuestros datos.

Veamos ahora qué ocurre si hacemos una búsqueda por cross-validation de los parámetros ideales. Queremos intentar mejorar lo que ya tenemos.

In [20]:
# Los parámetros por defecto usados han sido 

clf.get_params() 

# dónde gamma= 'scale' viene definida por la siguiente fórmula: gamma = 1/(464*Xtr.var())

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': True}

In [21]:
1/(464*Xtr.var())

4.680057982262105e-06

In [22]:
# Buscamos los parámetros óptimos con GridSearchCV

gammas = np.logspace(-7, 7, 10)
Cs = np.logspace(-2, 4, 10)  
tuned_parameters = { 'gamma': gammas,'C': Cs} 

clf2 = GridSearchCV(svm.SVC(kernel='rbf'), tuned_parameters, cv=5,n_jobs=-1,verbose=0)
clf2.fit(Xtr,ytr)
clf2=clf2.best_estimator_

score_tr = clf2.score(Xtr,ytr)
score_ts = clf2.score(Xts,yts)
preds_train=clf2.predict(Xtr)
preds_test = clf2.predict(Xts)

print('')
print('OA train %0.2f' % score_tr)
print('Kappa train %0.2f' % metrics.cohen_kappa_score(ytr,preds_train))
print('OA test %0.2f' % score_ts)
print('Kappa test %0.2f' % metrics.cohen_kappa_score(yts,preds_test))


OA train 0.87
Kappa train 0.84
OA test 0.60
Kappa test 0.52


In [37]:
clf2

SVC(C=4.6415888336127775, gamma=3.5938136638046257e-06)

In [23]:
clf2.get_params()

{'C': 4.6415888336127775,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 3.5938136638046257e-06,
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [24]:
# Matriz de confusión

CM=metrics.confusion_matrix(yts,preds_test)
print(CM)

[[ 7  2  1  1  2  4]
 [ 0 14  0  0  2  4]
 [ 0  0 12  3  2  3]
 [ 1  0  4 15  0  2]
 [ 3  1  0  1  9  4]
 [ 1  1  0  0  1  8]]


En general, vemos como al buscar los parámetros óptimos de la función Kernel (='rbf'), los resultados mejoran en todos los aspectos. Obtenemos un mayor acierto tanto en entrenamiento como en test y están balanceados, por lo que en principio no sospecharíamos de sobreajuste. 

Además, al ver la forma de la matriz de confusión volvemos a ver lo mismo, los resultados están muy balanceados.

Ahora vamos a probar a usar el conjunto que hemos obtenido tras eliminar algunas características que no tienen demasiada importancia. Dicho conjunto lo hemos obtenido en el notebook de Árboles. 

Lo cargamos y vemos si mejoran los resultados, o si simplemente no empeoran.

In [25]:
X_reduc2=np.load("./datos_mfcc_pca99_X_reducido2.npy")

In [26]:
# Ahora ya podemos dividir nuestro conjunto reducido en entrenamiento y test, y probar algunos métodos Kernel.

Xtr, Xts, ytr, yts = train_test_split(X_reduc2, y_reduc, test_size=0.3, shuffle=True, random_state=0)

In [33]:
# Buscamos los parámetros óptimos con GridSearchCV

gammas = np.logspace(-7, 7, 10)
Cs = np.logspace(-2, 4, 10)  
tuned_parameters = { 'gamma': gammas,'C': Cs} 

clf2 = GridSearchCV(svm.SVC(kernel='rbf'), tuned_parameters, cv=5,n_jobs=-1,verbose=0)
clf2.fit(Xtr,ytr)
clf2=clf2.best_estimator_

score_tr = clf2.score(Xtr,ytr)
score_ts = clf2.score(Xts,yts)
preds_train=clf2.predict(Xtr)
preds_test = clf2.predict(Xts)

print('')
print('OA train %0.2f' % score_tr)
print('Kappa train %0.2f' % metrics.cohen_kappa_score(ytr,preds_train))
print('OA test %0.2f' % score_ts)
print('Kappa test %0.2f' % metrics.cohen_kappa_score(yts,preds_test))


OA train 0.70
Kappa train 0.64
OA test 0.57
Kappa test 0.49


In [34]:
clf2.get_params()

{'C': 4.6415888336127775,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 3.5938136638046257e-06,
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [35]:
# Matriz de confusión

CM=metrics.confusion_matrix(yts,preds_test)
print(CM)

[[ 6  3  0  3  2  3]
 [ 0 12  0  0  2  6]
 [ 0  1  9  5  2  3]
 [ 1  0  3 16  0  2]
 [ 1  2  0  0 10  5]
 [ 1  0  1  0  0  9]]


Observamos que el rendimiento del clasificador ha empeorado un poco.

In [36]:
clf2

SVC(C=4.6415888336127775, gamma=3.5938136638046257e-06)